## Train MM / explore


In [1]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools

# default
room_size = "s"
capacity_max = 6
batch_size = 32
terminates_at = 19
num_iterations = (terminates_at + 1) * 20
validation_starts_at = num_iterations // 2

prob_type = (
    "non-equal-object-probs" if "different-prob" in room_size else "equal-object-probs"
)
root_path = (
    f"./training-results/{prob_type}/dqn/room_size={room_size}/capacity={capacity_max}/"
)


# Number of combinations you want
num_combinations = 100  # Change this to however many combinations you need

# random
test_seed_ = [i for i in range(num_combinations)]
target_update_interval_ = [5, 10, 20]
min_epsilon_ = [0.1, 0.5]
gamma_mm_ = [0.99, 0.9]
gamma_explore_ = [0.99, 0.9]
semantic_decay_factor_ = [0.5, 0.7, 0.9, 0.99]
agent_capacity_ = [
    {"episodic": capacity_max // 2, "semantic": capacity_max // 2, "short": 1},
    # {"episodic": capacity_max, "semantic": 0, "short": 1},
    # {"episodic": 0, "semantic": capacity_max, "short": 1},
]
replay_buffer_size_ = [
    num_iterations,
    num_iterations // 2,
]
warm_start_ = [
    num_iterations // 2,
    num_iterations // 5,
    num_iterations // 10,
]

# Generate all combinations
params_all = list(
    itertools.product(
        test_seed_,
        target_update_interval_,
        min_epsilon_,
        gamma_mm_,
        gamma_explore_,
        semantic_decay_factor_,
        agent_capacity_,
        replay_buffer_size_,
        warm_start_,
    )
)

# Get random combinations
random_combinations = random.sample(params_all, num_combinations)

for (
    test_seed,
    target_update_interval,
    min_epsilon,
    gamma_mm,
    gamma_explore,
    semantic_decay_factor,
    capacity,
    replay_buffer_size,
    warm_start,
) in tqdm(random_combinations):
    params = {
        "env_str": "room_env:RoomEnv-v2",
        "num_iterations": num_iterations,
        "replay_buffer_size": replay_buffer_size,
        "validation_starts_at": validation_starts_at,
        "warm_start": warm_start,
        "batch_size": batch_size,
        "target_update_interval": target_update_interval,
        "epsilon_decay_until": num_iterations,
        "max_epsilon": 1.0,
        "min_epsilon": min_epsilon,
        "gamma": {"mm": gamma_mm, "explore": gamma_explore},
        "capacity": capacity,
        "pretrain_semantic": False,
        "semantic_decay_factor": semantic_decay_factor,
        "lstm_params": {
            "num_layers": 2,
            "embedding_dim": 16,
            "hidden_size": 16,
            "bidirectional": False,
            "max_timesteps": terminates_at + 1,
            "max_strength": terminates_at + 1,
            "relu_for_attention": True,
        },
        "mlp_params": {
            "hidden_size": 16,
            "num_hidden_layers": 1,
            "dueling_dqn": True,
        },
        "num_samples_for_results": {"val": 5, "test": 10},
        "validation_interval": 5,
        "plotting_interval": 50,
        "train_seed": test_seed + 5,
        "test_seed": test_seed,
        "device": "cpu",
        "qa_function": "episodic_semantic",
        "explore_policy_heuristic": "avoid_walls",
        "env_config": {
            "question_prob": 1.0,
            "terminates_at": terminates_at,
            "randomize_observations": "objects",
            "room_size": room_size,
            "rewards": {"correct": 1, "wrong": 0, "partial": 0},
            "make_everything_static": False,
            "num_total_questions": 1000,
            "question_interval": 1,
            "include_walls_in_observations": True,
        },
        "ddqn": True,
        "default_root_dir": root_path,
    }

    agent = DQNAgent(**params)
    agent.train()

/home/tk/repos/agent-room-env-v2-lstm/agent/dqn/utils.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
